Procesamiento de datos del segundo fomulario. El que hice en Exactas y Fadu con aprox 15 personas en casa uno. 

Los datos originales estan aca:

https://github.com/IoniUTDT/Magia/tree/master/PilotoFacuFadu

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
# from scripts.constants_Exp2 import *
from scripts.preprocesamiento import *

path = "./PilotoFacuFadu/"
dataFile = "Data_Exp2.csv"

In [2]:
data = pd.read_csv(path+dataFile)
tabla = tablaPreprocesada(data, None)

/usr/lib/python3/dist-packages/pandas/core/indexing.py:346: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
  self.obj[item] = s


In [3]:
tabla.head()

,MecanismoPost,HayTruco,RelevanciaMano,ConfianzaPost,Edad,ConfianzaInicial,Profesion,MecanismoInicial,Formacion,MecanismoInicialOtros,IdExp,IdForm,IdSubset
0,No lo se,Si,3,1,18 a 24,5,Mate/Comp/Sistemas,Calor,Secundaria,"[Magnetismo, Presion]",1002,['31256'],None
1,Electricidad,Si,2,5,18 a 24,5,NaN,Calor,NaN,[Magnetismo],1002,['59406'],None
2,Magnetismo,Si,4,3,18 a 24,0,Ingenierias,Magnetismo,UniversidadCurso,"[Electricidad, Presion, Calor]",1002,['55654'],None
3,Presion,Si,1,7,25 a 40,6,Bio/Qui/Geo/Paleo,Presion,UniversidadCurso,[CampoLibre],1002,['15766'],None
4,Presion,Si,4,5,18 a 24,3,Bio/Qui/Geo/Paleo,Magnetismo,NaN,[Presion],1002,['61935'],None


In [4]:
#from pandas import ExcelWriter

tabla['GradoExactas'] = None
tabla['GradoExactas'][tabla['Profesion'].isin(L_Exactas)] = 'Exactas'
tabla['GradoExactas'][tabla['Profesion'].isin(L_Fisicas)] = 'Fisicas'
tabla['GradoExactas'][tabla['Profesion'].isin(L_Afines)] = 'Afines'
tabla['GradoExactas'][tabla['Profesion'].isin(L_NoAfines)] = 'NoAfines'
#tabla['GradoExactas'] = np.where(tabla['Profesion'].isin(L_Afines), 'Afin', None)
#tabla['GradoExactas'] = np.where(tabla['Profesion'].isin(L_NoAfines), 'NoAfines', None)
tabla
#writer = ExcelWriter(path+'TablaProcesada.xlsx')
#writer = pd.ExcelWriter(path+'TablaProcesada.xlsx', engine='xlsxwriter')
tabla.to_csv(path+'TablaProcesada.csv', encoding='utf-8')
#writer.save()

In [14]:
y='ConfianzaPost'
x='MecanismoPost'
hue='ConfianzaPost'
plot = sns.countplot(data=tabla, x=x, hue=hue)
plt.show()
plot.get_figure().savefig(path + "figures/count" + y +"Vs" + x + "hue" + hue)

#plt.savefig(path + "figures/" + "ConfianzaInicialVsMecanismoInicial")
#sns.barplot(x="Mecanismo", y="Confianza", data=tabla)
#plt.show()

In [25]:
conteo = tabla['GradoExactas'].value_counts()
display(conteo.index)
x='GradoExactas'
y='RelevanciaMano'
plot = sns.barplot(x=x, y=y, data=tabla, order=conteo.index)

conteo = conteo / conteo.max()

# Loop over the bars, and adjust the width (and position, to keep the bar centred)
for bar,newwidth in zip(plot.patches,conteo.tolist()):
    x_pos = bar.get_x()
    width = bar.get_width()
    centre = x_pos+width/2.

    bar.set_x(centre-newwidth/2.)
    bar.set_width(newwidth)

plt.show()
plot.get_figure().savefig(path + "figures/bar" + y +"Vs" + x)


Index(['Exactas', 'Afines', 'NoAfines'], dtype='object')

Una vez que tenemos la informacion cruda cargada la procesamos para transformarla en un dataframe legible. Eso significa pasar de todos los codigos numerico donde cada columna corresponde a un checkbox y esta en formato true/false (salvo las ecaladas numericas que ya vienen mejor procesadas) a un conjunto de columnas bien etiquetadas por categoria de pregunta con su respuesta.

In [8]:
tabla.columns




Index(['ConfianzaPost', 'MecanismoInicial', 'ConfianzaInicial', 'Formacion', 'Edad', 'Profesion', 'RelevanciaMano', 'HayTruco', 'MecanismoPost', 'MecanismoInicialOtros', 'IdExp', 'IdForm', 'IdSubset', 'GradoExactas'], dtype='object')

In [81]:
def pepe():
    i=-1
    while i<100:
        i += 1
        yield i

lala = pepe()
for i in lala:
    print(i)

0
1
2
3
4
5
6
7
8
9
10


In [87]:
pepe = {1:2, 3:4}
for key in list(pepe):
    print(key)
    
print(range(100))

1
3
range(0, 100)


In [71]:
pepe = lambda x: x**2
pepe(2)

4

In [14]:
# Transformamos las profesiones en una columna y agregamos si es exacta, afin o no. 

profesion = data[list(C_Profesion.keys())]
profesion = profesion.stack()
display (profesion.head())
profesion = profesion[profesion==1]
display (profesion.head())
profesion = profesion.reset_index()
profesion = profesion.rename (columns = {'level_0':'IdSujeto','level_1':'Profesion'})
profesion = profesion.replace({'Profesion':C_Profesion})
display (profesion.head())
#data_Konex = data_Konex[data_Konex.sum(axis=1) - data_Konex["Confianza"] == 3] # Esta linea elimina a todos los que no marcaron una y solo una de las opciones como correctas en cada pregunta
# data_Konex = data_Konex[data_Konex[list(dict_KonexFormacion.values())].sum(axis=1) == 1] # Esta linea elimina a todos los que no marcaron una y solo una de las opciones como correctas en cada pregunta
# data_Konex = data_Konex[data_Konex[list(dict_KonexExplicaciones.values())].sum(axis=1) == 1] # Esta linea elimina a todos los que no marcaron una y solo una de las opciones como correctas en cada pregunta
# data_Konex = data_Konex[data_Konex[list(dict_KonexTruco.values())].sum(axis=1) == 1] # Esta linea elimina a todos los que no marcaron una y solo una de las opciones como correctas en cada pregunta

# pepe = data_Konex.stack()
# pepe = pepe[(pepe==1) | (pepe.index.get_level_values(1)=="Confianza")]
# pepe = pepe.reset_index()
# pepe = pepe.rename(columns={'level_0':'IdSujeto','level_1':'valorElegido'})

# pepe["NombreColumna"] = pepe.index.map(lambda x: x % 4)
# pepe['NombreColumna'].replace([0,1,2,3],['Formacion','Mecanismo','Confianza','Truco'], inplace=True)
# condicion = pepe['NombreColumna']=='Confianza'
# pepe.loc[condicion,['valorElegido']] = pepe.loc[condicion,[0]].values 

# TablaFinal = pepe.pivot(index="IdSujeto", columns="NombreColumna", values="valorElegido")

0  1_2_8     0
   1_2_14    0
   1_2_11    0
   1_2_20    0
   1_2_2     0
dtype: int64

0  1_2_9     1
1  1_2_11    1
   1_2_4     1
2  1_2_15    1
3  1_2_1     1
dtype: int64

,IdSujeto,Profesion,0
0,0,Mate/Comp/Sistemas,1
1,1,Artes Plasticas,1
2,1,Arq/Diseño,1
3,2,Ingenierias,1
4,3,Bio/Qui/Geo/Paleo,1


In [5]:
data.columns

Index(['questionnaire_id', 'global_id', '1_1_0', '1_1_1', '1_1_2', '1_1_3', '1_1_4', '1_1_5', '1_2_0', '1_2_1', '1_2_2', '1_2_3', '1_2_4', '1_2_5', '1_2_6', '1_2_7', '1_2_8', '1_2_9', '1_2_10', '1_2_11', '1_2_12', '1_2_13', '1_2_14', '1_2_15', '1_2_16', '1_2_17', '1_2_18', '1_2_19', '1_2_20', '1_3_0', '1_3_1', '1_3_2', '1_3_3', '1_3_4', '1_3_5', '2_1_0', '2_1_1', '2_1_2', '2_1_3', '2_1_4', '2_1_5', '2_1_6', '2_1_7', '2_1_8', '2_2', '2_3_0', '2_3_1', '2_3_2', '2_3_3', '2_3_4', '2_3_5', '2_3_6', '2_3_7', '2_3_8', '2_4', '3_1_0', '3_1_1', '3_1_2', '3_1_3', '3_1_4', '3_1_5', '3_1_6', '3_1_7', '3_1_8', '3_2', '3_3_0', '3_3_1', '3_3_2', '4_1_1_0', '4_1_1_1', '4_1_1_2', '4_1_1_3', '4_1_1_4', '4_1_2_0', '4_1_2_1', '4_1_2_2', '4_1_2_3', '4_1_2_4', '4_1_3_0', '4_1_3_1', '4_1_3_2', '4_1_3_3', '4_1_3_4', '4_1_4_0', '4_1_4_1', '4_1_4_2', '4_1_4_3', '4_1_4_4', '4_1_5_0', '4_1_5_1', '4_1_5_2', '4_1_5_3', '4_1_5_4', '4_1_6_0', '4_1_6_1', '4_1_6_2', '4_1_6_3', '4_1_6_4', '4_1_7_0', '4_1_7_1', ...], dty